In [262]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import json,requests
import folium
from folium.plugins import MeasureControl
from folium.plugins import HeatMap

%matplotlib

Using matplotlib backend: MacOSX


In [2]:
client=MongoClient("mongodb://localhost:27017/")
db=client.companies

In [3]:
companies=db.companies.find()

In [4]:
datos= pd.DataFrame(companies)

In [5]:
datos.head()

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8f,Omnidrive,omnidrive,http://www.crunchbase.com/company/omnidrive,http://www.omnidrive.com,http://www.omnidrive.com/blog,http://feeds.feedburner.com/omnidrive,Nomadesk,network_hosting,NaN,...,[],[],[],[],11.0,1.0,9.0,15.0,,None
2,52cdef7c4bab8bd675297d92,Flektor,flektor,http://www.crunchbase.com/company/flektor,http://www.flektor.com,http://www.flektor-blog.com,http://www.flektor-blog.com/video_editing_soft...,None,games_video,NaN,...,[],[],[],[],NaN,NaN,NaN,NaN,None,None
3,52cdef7c4bab8bd675297d90,Postini,postini,http://www.crunchbase.com/company/postini,http://postini.com,,,None,web,NaN,...,[],[],[],[],6.0,2.0,NaN,NaN,None,None
4,52cdef7c4bab8bd675297d91,Geni,geni,http://www.crunchbase.com/company/geni,http://www.geni.com,http://blog.geni.com,http://blog.geni.com/index.rdf,geni,web,18.0,...,"[{'embed_code': '<object width=""425"" height=""3...",[],[],[],6.0,1.0,NaN,NaN,None,None


In [6]:
#viendo tipo de datos.
datos["category_code"].value_counts()

web                 3787
software            2736
games_video         1083
mobile              1018
other                986
advertising          928
enterprise           742
ecommerce            688
consulting           637
network_hosting      626
public_relations     533
search               394
biotech              373
hardware             368
cleantech            305
semiconductor        167
security             156
analytics             66
finance               49
social                49
news                  48
education             36
music                 33
messaging             30
legal                 25
travel                25
medical               25
health                23
photo_video           23
manufacturing         19
sports                13
real_estate           10
fashion               10
automotive             9
hospitality            8
transportation         7
nanotech               5
design                 4
nonprofit              4
local                  1


In [7]:
query = db.companies.find({"$and":[
                               {"founded_year": {"$gte": 1990}},
                               {"number_of_employees": {"$gte": 7}},
                                {"total_money_raised":{"$regex":"M"}},
                               {"$or": [
                                   {"category_code": "design" } ,
                                   {"category_code": "biotech" } ,
                                   {"category_code":"games_video"},
                                   {"category_code":"web"},
                                   {"category_code": "software" } ,
                                   {"category_code": "education" } ,
                                   ]}
                               ],
                               },
                                {"_id": 0, "crunchbase_url": 0, "products": 0, "founded_month": 0, "founded_day": 0,
                                "acquisition": 0, "acquisitions": 0, "video_embeds": 0,
                                "screenshots": 0, "external_links": 0, "partners": 0,
                                "image": 0, "blog_feed_url": 0, "blog_url":0, "deadpooled_day":0,
                                "deadpooled_year":0, "deadpooled_month":0, "deadpooled_url":0, "ipo":0,
                                "permalink":0, "homepage_url":0, "tag_list":0, "email_addres": 0, "providerships": 0,
                                "milestones": 0, "twitter_username": 0, "updated_at": 0, "email_address":0, "investments":0,
                               }
                                    )

In [8]:
datos= pd.DataFrame(query)

In [9]:
datos.head()

,name,category_code,number_of_employees,founded_year,alias_list,phone_number,description,created_at,overview,relationships,competitions,total_money_raised,funding_rounds,offices
0,Geni,web,18,2006,,,Geneology social network site,Thu May 31 19:52:34 UTC 2007,<p>Geni is an online community of casual and e...,"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Ancestry', 'permalin...",$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '..."
1,Wetpaint,web,47,2005,,206.859.6300,Technology Platform Company,2007-05-25 06:51:27,<p>Wetpaint is a technology platform company t...,"[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...","[{'description': '', 'address1': '710 - 2nd Av..."
2,Plaxo,web,50,2002,,650-254-5400,Contact Management,Sat Jun 09 14:18:54 UTC 2007,<p>Plaxo helps keep people connected by solvin...,"[{'is_past': False, 'title': 'President & CEO'...","[{'competitor': {'name': 'Xobni', 'permalink':...",$28.3M,"[{'id': 20, 'round_code': 'a', 'source_url': '...","[{'description': 'HQ', 'address1': '1050 Enter..."
3,Mahalo,web,40,2007,,,Life long learning content,Thu Jun 14 03:42:20 UTC 2007,<p>Mahalo is a human powered search engine fou...,"[{'is_past': False, 'title': 'Founder, CEO', '...","[{'competitor': {'name': 'Topicle', 'permalink...",$21M,"[{'id': 323, 'round_code': 'a', 'source_url': ...","[{'description': '', 'address1': '3525 Eastham..."
4,Kyte,games_video,40,2006,,,Online & Mobile Video Platform,Thu Jun 14 18:26:11 UTC 2007,<p>Kyte is the online and mobile video platfor...,"[{'is_past': False, 'title': 'CTO and co-found...","[{'competitor': {'name': 'Ustream', 'permalink...",$23.4M,"[{'id': 32, 'round_code': 'a', 'source_url': '...","[{'description': None, 'address1': '442 Post S..."


In [10]:
datos.shape

(629, 14)

In [11]:
datos.isnull().sum()

name                     0
category_code            0
number_of_employees      0
founded_year             0
alias_list             148
phone_number             2
description             35
created_at               0
overview                 0
relationships            0
competitions             0
total_money_raised       0
funding_rounds           0
offices                  0
dtype: int64

In [12]:
datos.drop(columns=["alias_list","created_at"],inplace=True)

In [13]:
datos["description"]=datos["description"].fillna(0)

In [14]:
datos=datos.replace(np.nan,0)

In [15]:
datos.isnull().sum()

name                   0
category_code          0
number_of_employees    0
founded_year           0
phone_number           0
description            0
overview               0
relationships          0
competitions           0
total_money_raised     0
funding_rounds         0
offices                0
dtype: int64

In [16]:
datos.shape

(629, 12)

In [17]:
datos.dropna(how="all",subset={"offices"},inplace=True)

In [18]:
latitude = db.companies.find({},{"offices.latitude":1})
longitude = db.companies.find({},{"offices.longitude":1})


In [19]:
def geopoint(x):
   for ele in x:
       longitude = ele["longitude"]
       latitude = ele["latitude"]
       return{"type": "Point", "coordinates": {"longitude":longitude,"latitude":latitude}}
    
datos["geo"] = datos["offices"].apply(geopoint)


In [20]:
datos["geo"].value_counts()

{'type': 'Point', 'coordinates': {'longitude': None, 'latitude': None}}                  122
{'type': 'Point', 'coordinates': {'longitude': -122.419204, 'latitude': 37.775196}}        4
{'type': 'Point', 'coordinates': {'longitude': -95.712891, 'latitude': 37.09024}}          3
{'type': 'Point', 'coordinates': {'longitude': 34.7595, 'latitude': 32.0554}}              3
{'type': 'Point', 'coordinates': {'longitude': -122.5162892, 'latitude': 37.9260091}}      2
                                                                                        ... 
{'type': 'Point', 'coordinates': {'longitude': -122.0302814, 'latitude': 37.3183744}}      1
{'type': 'Point', 'coordinates': {'longitude': -122.206893, 'latitude': 37.484619}}        1
{'type': 'Point', 'coordinates': {'longitude': 13.38861, 'latitude': 52.4897}}             1
{'type': 'Point', 'coordinates': {'longitude': -71.255855, 'latitude': 42.393662}}         1
{'type': 'Point', 'coordinates': {'longitude': -122.411157, 'latitude'

In [21]:
a=datos[datos["geo"].isnull()].index

In [22]:
datos.drop(a,axis=0,inplace=True)

In [23]:
datos["geo"].isnull().sum()

0

In [24]:
b=datos[datos["geo"]=={'type': 'Point', 'coordinates': {'longitude': None, 'latitude': None}}].index
datos.drop(b,axis=0,inplace=True)

In [25]:
datos["geo"].value_counts()

{'type': 'Point', 'coordinates': {'longitude': -122.419204, 'latitude': 37.775196}}      4
{'type': 'Point', 'coordinates': {'longitude': 34.7595, 'latitude': 32.0554}}            3
{'type': 'Point', 'coordinates': {'longitude': -95.712891, 'latitude': 37.09024}}        3
{'type': 'Point', 'coordinates': {'longitude': -122.4007846, 'latitude': 37.7899027}}    2
{'type': 'Point', 'coordinates': {'longitude': -122.135282, 'latitude': 47.582658}}      2
                                                                                        ..
{'type': 'Point', 'coordinates': {'longitude': -96.823778, 'latitude': 32.969251}}       1
{'type': 'Point', 'coordinates': {'longitude': -89.39256, 'latitude': 43.068006}}        1
{'type': 'Point', 'coordinates': {'longitude': -122.206893, 'latitude': 37.484619}}      1
{'type': 'Point', 'coordinates': {'longitude': -0.0934254, 'latitude': 51.5141566}}      1
{'type': 'Point', 'coordinates': {'longitude': -122.10649, 'latitude': 37.431409}}       1

In [26]:
datos.to_json(r'./oficina.json', orient="records")

In [202]:
#Volvemos a traer la coleccion creada de Mongo
db = client.companies
consulta=db.indice.find()

In [203]:
cercania = pd.DataFrame(consulta)

In [204]:
cercania.head()

,_id,name,category_code,number_of_employees,founded_year,phone_number,description,overview,relationships,competitions,total_money_raised,funding_rounds,offices,geo
0,5d866b608b86b2a8702af75c,Pando Networks,games_video,23,2004,212-343-8800,Accelerating large media delivery,<p>Pando Networks (www.pandonetworks.com) impr...,"[{'is_past': False, 'title': 'Founder & CEO', ...",[],$11M,"[{'id': 1750, 'round_code': 'a', 'source_url':...","[{'description': None, 'address1': '520 Broadw...","{'type': 'Point', 'coordinates': {'longitude':..."
1,5d866b608b86b2a8702af75d,Jangl SMS,web,22,2005,,,<p>Jangl allows consumers to exchange text mes...,"[{'is_past': True, 'title': 'Founder, CTO', 'p...","[{'competitor': {'name': 'SayNow', 'permalink'...",$9M,"[{'id': 48, 'round_code': 'b', 'source_url': '...","[{'description': '', 'address1': '6601 Owens D...","{'type': 'Point', 'coordinates': {'longitude':..."
2,5d866b608b86b2a8702af75e,AdaptiveBlue,games_video,15,2007,,,<p>AdaptiveBlue is the company that develops G...,"[{'is_past': False, 'title': 'COO', 'person': ...","[{'competitor': {'name': 'Inveni', 'permalink'...",$24M,"[{'id': 50, 'round_code': 'a', 'source_url': '...","[{'description': '', 'address1': '131 Varick S...","{'type': 'Point', 'coordinates': {'longitude':..."
3,5d866b608b86b2a8702af75f,Mahalo,web,40,2007,,Life long learning content,<p>Mahalo is a human powered search engine fou...,"[{'is_past': False, 'title': 'Founder, CEO', '...","[{'competitor': {'name': 'Topicle', 'permalink...",$21M,"[{'id': 323, 'round_code': 'a', 'source_url': ...","[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': {'longitude':..."
4,5d866b608b86b2a8702af760,SodaHead,web,25,2007,1.818.981.8476,Online Community and Discussion,"<p>SodaHead Inc., provides best in class polli...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Toluna', 'permalink'...",$12.7M,"[{'id': 71, 'round_code': 'a', 'source_url': '...","[{'description': '', 'address1': '16161 Ventur...","{'type': 'Point', 'coordinates': {'longitude':..."


In [205]:
cercania.shape

(501, 14)

In [206]:
cercania["offices"].value_counts()

[{'description': '', 'address1': '116 New Montgomery', 'address2': '', 'zip_code': '94105', 'city': 'San Francisco', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.7870923, 'longitude': -122.3999719}]                                                                                                                                                                                                                                                                   2
[{'description': 'Headquarters', 'address1': '333 Twin Dolphin Drive', 'address2': '', 'zip_code': '94065', 'city': 'Redwood City', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.522281, 'longitude': -122.258139}]                                                                                                                                                                                                                                                      2
[{'description': 'Koofers', 'address1': '', 'address2': 

In [207]:
ciudades=[e[0]["city"] for e in cercania["offices"]]

In [208]:
len(ciudades)

501

In [209]:
cercania["ciudades"]=ciudades

In [213]:
def gps (eg):
    cercania["longitude"]=[e["coordinates"]["longitude"] for e in cercania["geo"]]
    cercania["latitude"]=[e["coordinates"]["latitude"] for e in cercania["geo"]]
    return 

In [214]:
cercania["geo"].apply(gps)

0      None
1      None
2      None
3      None
4      None
       ... 
496    None
497    None
498    None
499    None
500    None
Name: geo, Length: 501, dtype: object

In [215]:
display(cercania)

,_id,name,category_code,number_of_employees,founded_year,phone_number,description,overview,relationships,competitions,total_money_raised,funding_rounds,offices,geo,ciudades,longitude,latitude
0,5d866b608b86b2a8702af75c,Pando Networks,games_video,23,2004,212-343-8800,Accelerating large media delivery,<p>Pando Networks (www.pandonetworks.com) impr...,"[{'is_past': False, 'title': 'Founder & CEO', ...",[],$11M,"[{'id': 1750, 'round_code': 'a', 'source_url':...","[{'description': None, 'address1': '520 Broadw...","{'type': 'Point', 'coordinates': {'longitude':...",New York,-73.998730,40.722655
1,5d866b608b86b2a8702af75d,Jangl SMS,web,22,2005,,,<p>Jangl allows consumers to exchange text mes...,"[{'is_past': True, 'title': 'Founder, CTO', 'p...","[{'competitor': {'name': 'SayNow', 'permalink'...",$9M,"[{'id': 48, 'round_code': 'b', 'source_url': '...","[{'description': '', 'address1': '6601 Owens D...","{'type': 'Point', 'coordinates': {'longitude':...",Pleasanton,-121.907768,37.697805
2,5d866b608b86b2a8702af75e,AdaptiveBlue,games_video,15,2007,,,<p>AdaptiveBlue is the company that develops G...,"[{'is_past': False, 'title': 'COO', 'person': ...","[{'competitor': {'name': 'Inveni', 'permalink'...",$24M,"[{'id': 50, 'round_code': 'a', 'source_url': '...","[{'description': '', 'address1': '131 Varick S...","{'type': 'Point', 'coordinates': {'longitude':...",NYC,-74.337200,40.801358
3,5d866b608b86b2a8702af75f,Mahalo,web,40,2007,,Life long learning content,<p>Mahalo is a human powered search engine fou...,"[{'is_past': False, 'title': 'Founder, CEO', '...","[{'competitor': {'name': 'Topicle', 'permalink...",$21M,"[{'id': 323, 'round_code': 'a', 'source_url': ...","[{'description': '', 'address1': '3525 Eastham...","{'type': 'Point', 'coordinates': {'longitude':...",Culver City,-118.487267,34.017606
4,5d866b608b86b2a8702af760,SodaHead,web,25,2007,1.818.981.8476,Online Community and Discussion,"<p>SodaHead Inc., provides best in class polli...","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Toluna', 'permalink'...",$12.7M,"[{'id': 71, 'round_code': 'a', 'source_url': '...","[{'description': '', 'address1': '16161 Ventur...","{'type': 'Point', 'coordinates': {'longitude':...",Encino,-119.306607,37.269175
5,5d866b608b86b2a8702af761,Geni,web,18,2006,,Geneology social network site,<p>Geni is an online community of casual and e...,"[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Ancestry', 'permalin...",$16.5M,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...","{'type': 'Point', 'coordinates': {'longitude':...",West Hollywood,-118.393064,34.090368
6,5d866b608b86b2a8702af762,Wakoopa,web,8,2007,,Social network for desktop software,<p>Wakoopa creates digital DNA of today&#8217;...,"[{'is_past': False, 'title': 'Co-founder', 'pe...","[{'competitor': {'name': 'RescueTime', 'permal...",$1M,"[{'id': 100, 'round_code': 'a', 'source_url': ...","[{'description': 'Arts & Crafts', 'address1': ...","{'type': 'Point', 'coordinates': {'longitude':...",Amsterdam,4.894862,52.374523
7,5d866b608b86b2a8702af763,Coghead,web,21,2003,0,0,<p>Coghead is a WYSIWYG database driven applic...,"[{'is_past': True, 'title': 'Head of Marketing...","[{'competitor': {'name': 'LongJump', 'permalin...",$11.2M,"[{'id': 102, 'round_code': 'a', 'source_url': ...","[{'description': None, 'address1': None, 'addr...","{'type': 'Point', 'coordinates': {'longitude':...",Redwood City,-122.221717,37.476479
8,5d866b608b86b2a8702af764,Wetpaint,web,47,2005,206.859.6300,Technology Platform Company,<p>Wetpaint is a technology platform company t...,"[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...","[{'description': '', 'address1': '710 - 2nd Av...","{'type': 'Point', 'coordinates': {'longitude':...",Seattle,-122.333253,47.603122
9,5d866b608b86b2a8702af765,Livestream,games

In [236]:
def buscador (ciudad):
    a=cercania[cercania["ciudades"]==ciudad].index
    centrar=cercania.iloc[a,14:17]
    return centrar

In [237]:
b=buscador("New York")


In [244]:
b

,ciudades,longitude,latitude
0,New York,-73.998730,40.722655
9,New York,-73.995625,40.726155
11,New York,-73.995722,40.726040
45,New York,-95.712891,37.090240
47,New York,-74.004792,40.707921
52,New York,-74.004592,40.744060
64,New York,-74.001704,40.723384
88,New York,-77.095965,38.984172
104,New York,-73.969296,40.762673
105,New York,-122.117418,37.402802


In [74]:
#API Starbucks en New York
url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
 client_id='BK1LUEVPQKUICZTO2Y2ZX1XZLFLTPD0AJVMXXPOT04AQ1W5P',
 client_secret='IPWJ0UWZKWC11ZKL3Z2JUAGRMEX2TMMJJQOO3JXE0Q4K5FT1',
 v='20180323',
 ll='40.7243,-74.0018',
 query='Starbucks',
)
resp = requests.get(url=url, params=params)
data_api = json.loads(resp.text)

In [75]:
display(data_api)

{'meta': {'code': 200, 'requestId': '5d87eb306bdee60039d083b5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'SoHo',
  'headerFullLocation': 'SoHo, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'starbucks',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 40.743224295893356, 'lng': -73.9899953035},
   'sw': {'lat': 40.70884472813157, 'lng': -74.0166453665}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '516409d5e4b0ec7be4b67ab4',
       'name': 'Starbucks',
       'contact': {},
       'location': {'address': '370 Canal St',
        'crossStreet': 'at Sheraton Tribeca',
        'l

In [239]:
def cafe(lugar):
  nombre=[]
  latitud=[]
  longi=[]
  for e in data_api['response']['groups'][0]['items']:
      nombre.append(e['venue']['name'])
      latitud.append(e['venue']['location']['lat'])
      longi.append(e['venue']['location']['lng'])
  lugar = {"name":nombre, "latitude":latitud, "longitude":longi}
  return pd.DataFrame(lugar)
cafe = cafe(data_api)
cafe.head()

,name,latitude,longitude
0,Starbucks,40.720837,-74.004366
1,Starbucks Reserve,40.729943,-73.993913
2,Starbucks,40.726890,-74.000260
3,Starbucks,40.719852,-74.004950
4,Starbucks,40.722585,-73.998023


In [271]:
mapa= folium.Map(location=[40.722655, -73.998730], width=750, height=500, zoom_start=15)

In [272]:
for index, row in cafe.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=9,
                        popup="City: {}, latitude {}, longitude {}".format(row['name'], row['latitude'], row['longitude']),
                       ).add_to(mapa)
    folium.Marker([40.722655, -73.998730],
                        radius=2,
                        icon=folium.Icon(icon='cloud'),
                       ).add_to(mapa)
    folium.Circle(location=[40.722655, -73.998730],
                    radius=500
                   ).add_to(mapa)
mapa.add_child(MeasureControl())

In [ ]:
#Despues de muchas vueltas no he conseguido sacar el near. A ver si en un hueco me lo podeis ayudar. Muchas gracias
#dejo la parte en sucio porque me gustaria poder volver a el.


In [173]:
'''''def geocerca(geopoint, maxdistance=1000):
    return db.cercania.find({
        "geo":{
            "$near":{
                "$geometry":geopoint,
                "$maxDistance":maxdistance
            }}})'''

In [274]:
#san = pd.DataFrame(geonear(b, 5000))

In [ ]:
'''''''
{
   "geo": {
     "$near": {
       "$geometry": {
          type: "Point" ,
          coordinates: [ <longitude> , <latitude> ]
       },
       "$maxDistance": 2000
     }
   }
}
''''''